In [1]:
import sys

rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')


from atom import Atom

# this dictionary is used to deal with the annotated entities in texts.
fLabel = {'症状和体征': 'Sy',
          '身体部位': 'Bo',
          '检查和检验': 'Ch',
          '治疗': 'Tr',
          '疾病和诊断': 'Si'}

def strQ2B(ustring):
    rstring = ''
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):
            inside_code -= 65248
        # 2: unichr; 3: chr
        rstring += chr(inside_code)
    return rstring

def cleanSpace(text):
    #return text.replace("\t"," ").replace(" ","&nbsp;").replace('\n', '')
    '''
    return text.strip()
    '''
    for i in ' \t\r\x0b\x0c':
        text = text.replace(i, '@')
    text = text.strip()
    text =  text.replace('\n', '@')
    return text 
    
def segmentStr(text):
    strLen = len(text)
    atoms = []
    for i in range(strLen):
        atoms.append(text[i])
    return atoms

In [2]:
class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    
    # Section0: __init__
    def __init__(self, text):
        pass
    
    # Section1: Atom Tags
    def setAtomTags(self):
        pass
       
    # Section2: POS Tags
    def getPOSTags(self):
        pass
    
    def setPOSTags(self):
        pass
    
    def updateDF(self):
        pass
    
    def toTextFile(self, filePath, cols = None):
        pass
    
    # Section3: Annotated Text
    def coporAnnotation(self, newTag):
        pass
    
    def getAnnotedEntities(self, eType, annotedFilePath = None):
        pass
    
    # Section4: Predicted Result
    def coporResult(self, result, newTag):
        pass
    
    def getLearnedEntities(self, eType):
        pass 
           
    # Section5: Get Statistics
    def data(self):
        pass
    
    def execute(self):
        pass
    

### 0.  `__init__`

In [3]:
class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    entities = []
    learnedEntities = []
    annotedEntities = []
    learnedREntities = []
    annotedREntities = []
    annotedFilePath = None
    score = None
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)

### 1 Atom tags

In [4]:
with open('dev/Text/一般项目-1.txtoriginal.txt', 'r') as f:
    text = f.read()  
text

'女性，88岁，农民，双滦区应营子村人，主因右髋部摔伤后疼痛肿胀，活动受限5小时于2016-10-29；11：12入院。\n\n\n\n'

In [5]:
cct = ChineseClinicalText(text)
cct._text
length = len(cct._text)
strQ2B(text)[:length] == cct._text

True

In [6]:
cct.atomsBaseList

['女',
 '性',
 ',',
 '8',
 '8',
 '岁',
 ',',
 '农',
 '民',
 ',',
 '双',
 '滦',
 '区',
 '应',
 '营',
 '子',
 '村',
 '人',
 ',',
 '主',
 '因',
 '右',
 '髋',
 '部',
 '摔',
 '伤',
 '后',
 '疼',
 '痛',
 '肿',
 '胀',
 ',',
 '活',
 '动',
 '受',
 '限',
 '5',
 '小',
 '时',
 '于',
 '2',
 '0',
 '1',
 '6',
 '-',
 '1',
 '0',
 '-',
 '2',
 '9',
 ';',
 '1',
 '1',
 ':',
 '1',
 '2',
 '入',
 '院',
 '。']

In [7]:
cct.setAtomTags()

In [8]:
for a in cct.atoms:
    print(a)

女	CHN	OTHER	女
性	CHN	OTHER	忄
,	PUNC	OTHER	-
8	NUM	OTHER	-
8	NUM	OTHER	-
岁	CHN	OTHER	山
,	PUNC	OTHER	-
农	CHN	OTHER	冖
民	CHN	OTHER	氏
,	PUNC	OTHER	-
双	CHN	SYM_UNIT	又
滦	CHN	OTHER	氵
区	CHN	OTHER	匚
应	CHN	OTHER	广
营	CHN	OTHER	艹
子	CHN	OTHER	子
村	CHN	OTHER	木
人	CHN	OTHER	人
,	PUNC	OTHER	-
主	CHN	OTHER	丶
因	CHN	BREAK_UNIT	囗
右	CHN	SITE_UNIT	口
髋	CHN	PART_UNIT	骨
部	CHN	SITE_UNIT	阝
摔	CHN	OTHER	扌
伤	CHN	OTHER	亻
后	CHN	SITE_UNIT	口
疼	CHN	SYM_UNIT	疒
痛	CHN	SYM_UNIT	疒
肿	CHN	OTHER	月
胀	CHN	OTHER	月
,	PUNC	OTHER	-
活	CHN	OTHER	氵
动	CHN	OTHER	力
受	CHN	OTHER	又
限	CHN	OTHER	阝
5	NUM	OTHER	-
小	CHN	SYM_UNIT	小
时	CHN	OTHER	日
于	CHN	OTHER	二
2	NUM	OTHER	-
0	NUM	OTHER	-
1	NUM	OTHER	-
6	NUM	OTHER	-
-	PUNC	OTHER	-
1	NUM	OTHER	-
0	NUM	OTHER	-
-	PUNC	OTHER	-
2	NUM	OTHER	-
9	NUM	OTHER	-
;	PUNC	OTHER	-
1	NUM	OTHER	-
1	NUM	OTHER	-
:	PUNC	OTHER	-
1	NUM	OTHER	-
2	NUM	OTHER	-
入	CHN	OTHER	入
院	CHN	OTHER	阝
。	PUNC	OTHER	-


### 2 POS Tags

In [9]:
from jieba import posseg as ppseg

In [10]:
segs = list(ppseg.cut(cct._text))
segs

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/j0/r1vb1dcs195fd2gggjcmmlqm0000gp/T/jieba.cache
Loading model cost 0.996 seconds.
Prefix dict has been built succesfully.


[pair('女性', 'n'),
 pair(',', 'x'),
 pair('88', 'm'),
 pair('岁', 'm'),
 pair(',', 'x'),
 pair('农民', 'n'),
 pair(',', 'x'),
 pair('双滦区', 'nr'),
 pair('应', 'v'),
 pair('营子', 'n'),
 pair('村', 'zg'),
 pair('人', 'n'),
 pair(',', 'x'),
 pair('主因', 'n'),
 pair('右', 'f'),
 pair('髋部', 'n'),
 pair('摔伤', 'v'),
 pair('后', 'f'),
 pair('疼痛', 'n'),
 pair('肿胀', 'v'),
 pair(',', 'x'),
 pair('活动', 'vn'),
 pair('受限', 'v'),
 pair('5', 'x'),
 pair('小时', 'n'),
 pair('于', 'p'),
 pair('2016', 'm'),
 pair('-', 'x'),
 pair('10', 'm'),
 pair('-', 'x'),
 pair('29', 'm'),
 pair(';', 'x'),
 pair('11', 'm'),
 pair(':', 'x'),
 pair('12', 'm'),
 pair('入院', 'n'),
 pair('。', 'x')]

In [11]:
pair = segs[0]
pair.word

'女性'

In [12]:
for i in range(len(segs)):
    pair = segs[i]
    start = sum(len(p.word) for p in segs[:i])
    end = sum(len(p.word) for p in segs[:i+1]) -1
    pair.indeces = [start, end]
    # print( text[start: end +1], '\t',pair.flag, '\t',  pair.indeces)
    
## Test Results
for p in segs:
    print(p.word,'\t' , p.flag,'\t', p.indeces)

女性 	 n 	 [0, 1]
, 	 x 	 [2, 2]
88 	 m 	 [3, 4]
岁 	 m 	 [5, 5]
, 	 x 	 [6, 6]
农民 	 n 	 [7, 8]
, 	 x 	 [9, 9]
双滦区 	 nr 	 [10, 12]
应 	 v 	 [13, 13]
营子 	 n 	 [14, 15]
村 	 zg 	 [16, 16]
人 	 n 	 [17, 17]
, 	 x 	 [18, 18]
主因 	 n 	 [19, 20]
右 	 f 	 [21, 21]
髋部 	 n 	 [22, 23]
摔伤 	 v 	 [24, 25]
后 	 f 	 [26, 26]
疼痛 	 n 	 [27, 28]
肿胀 	 v 	 [29, 30]
, 	 x 	 [31, 31]
活动 	 vn 	 [32, 33]
受限 	 v 	 [34, 35]
5 	 x 	 [36, 36]
小时 	 n 	 [37, 38]
于 	 p 	 [39, 39]
2016 	 m 	 [40, 43]
- 	 x 	 [44, 44]
10 	 m 	 [45, 46]
- 	 x 	 [47, 47]
29 	 m 	 [48, 49]
; 	 x 	 [50, 50]
11 	 m 	 [51, 52]
: 	 x 	 [53, 53]
12 	 m 	 [54, 55]
入院 	 n 	 [56, 57]
。 	 x 	 [58, 58]


In [13]:
POSTagsList = []
for p in segs:
    word = p.word
    for i in range(len(p.word)):
        if i == 0:
            POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
        else:
            POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
POSTagsList

[[0, '女', 'n-B'],
 [1, '性', 'n-I'],
 [2, ',', 'x-B'],
 [3, '8', 'm-B'],
 [4, '8', 'm-I'],
 [5, '岁', 'm-B'],
 [6, ',', 'x-B'],
 [7, '农', 'n-B'],
 [8, '民', 'n-I'],
 [9, ',', 'x-B'],
 [10, '双', 'nr-B'],
 [11, '滦', 'nr-I'],
 [12, '区', 'nr-I'],
 [13, '应', 'v-B'],
 [14, '营', 'n-B'],
 [15, '子', 'n-I'],
 [16, '村', 'zg-B'],
 [17, '人', 'n-B'],
 [18, ',', 'x-B'],
 [19, '主', 'n-B'],
 [20, '因', 'n-I'],
 [21, '右', 'f-B'],
 [22, '髋', 'n-B'],
 [23, '部', 'n-I'],
 [24, '摔', 'v-B'],
 [25, '伤', 'v-I'],
 [26, '后', 'f-B'],
 [27, '疼', 'n-B'],
 [28, '痛', 'n-I'],
 [29, '肿', 'v-B'],
 [30, '胀', 'v-I'],
 [31, ',', 'x-B'],
 [32, '活', 'vn-B'],
 [33, '动', 'vn-I'],
 [34, '受', 'v-B'],
 [35, '限', 'v-I'],
 [36, '5', 'x-B'],
 [37, '小', 'n-B'],
 [38, '时', 'n-I'],
 [39, '于', 'p-B'],
 [40, '2', 'm-B'],
 [41, '0', 'm-I'],
 [42, '1', 'm-I'],
 [43, '6', 'm-I'],
 [44, '-', 'x-B'],
 [45, '1', 'm-B'],
 [46, '0', 'm-I'],
 [47, '-', 'x-B'],
 [48, '2', 'm-B'],
 [49, '9', 'm-I'],
 [50, ';', 'x-B'],
 [51, '1', 'm-B'],
 [52, '1', 'm-I'

In [14]:
def getPOSTagsList(text):
    segs = list(ppseg.cut(text))
    
    for i in range(len(segs)):
        pair = segs[i]
        start = sum(len(p.word) for p in segs[:i])
        end = sum(len(p.word) for p in segs[:i+1]) -1
        pair.indeces = [start, end]
        
    POSTagsList = []
    for p in segs:
        word = p.word
        for i in range(len(p.word)):
            if i == 0:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
            else:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
                
    return POSTagsList

In [15]:
getPOSTagsList(text)

[[0, '女', 'n-B'],
 [1, '性', 'n-I'],
 [2, '，', 'x-B'],
 [3, '8', 'm-B'],
 [4, '8', 'm-I'],
 [5, '岁', 'm-B'],
 [6, '，', 'x-B'],
 [7, '农', 'n-B'],
 [8, '民', 'n-I'],
 [9, '，', 'x-B'],
 [10, '双', 'nr-B'],
 [11, '滦', 'nr-I'],
 [12, '区', 'nr-I'],
 [13, '应', 'v-B'],
 [14, '营', 'n-B'],
 [15, '子', 'n-I'],
 [16, '村', 'zg-B'],
 [17, '人', 'n-B'],
 [18, '，', 'x-B'],
 [19, '主', 'n-B'],
 [20, '因', 'n-I'],
 [21, '右', 'f-B'],
 [22, '髋', 'n-B'],
 [23, '部', 'n-I'],
 [24, '摔', 'v-B'],
 [25, '伤', 'v-I'],
 [26, '后', 'f-B'],
 [27, '疼', 'n-B'],
 [28, '痛', 'n-I'],
 [29, '肿', 'v-B'],
 [30, '胀', 'v-I'],
 [31, '，', 'x-B'],
 [32, '活', 'vn-B'],
 [33, '动', 'vn-I'],
 [34, '受', 'v-B'],
 [35, '限', 'v-I'],
 [36, '5', 'x-B'],
 [37, '小', 'n-B'],
 [38, '时', 'n-I'],
 [39, '于', 'p-B'],
 [40, '2', 'm-B'],
 [41, '0', 'm-I'],
 [42, '1', 'm-I'],
 [43, '6', 'm-I'],
 [44, '-', 'x-B'],
 [45, '1', 'm-B'],
 [46, '0', 'm-I'],
 [47, '-', 'x-B'],
 [48, '2', 'm-B'],
 [49, '9', 'm-I'],
 [50, '；', 'x-B'],
 [51, '1', 'm-B'],
 [52, '1', 'm-I'

In [16]:
from pandas import read_csv
from pandas import DataFrame as df

class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    entities = []
    learnedEntities = []
    annotedEntities = []
    learnedREntities = []
    annotedREntities = []
    annotedFilePath = None
    score = None
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
    
    def updateDF(self):
        'update the attribute for pandas PF form sentence'
        DF = df([i.to_dict() for i in self.atoms])
        # TODO, may not contain POS Tag
        # OK, actually, it can. hh
        self.dfFormat = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags]
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


In [17]:
text

'女性，88岁，农民，双滦区应营子村人，主因右髋部摔伤后疼痛肿胀，活动受限5小时于2016-10-29；11：12入院。\n\n\n\n'

In [18]:
cct = ChineseClinicalText(text)
cct._text

'女性,88岁,农民,双滦区应营子村人,主因右髋部摔伤后疼痛肿胀,活动受限5小时于2016-10-29;11:12入院。'

In [19]:
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()

In [20]:
a = cct.atoms[0]
a.to_dict()

{'POSTag': 'n-B',
 '_atom': '女',
 'basicTag': 'CHN',
 'dictTag': 'OTHER',
 'index': 0,
 'radical': '女'}

In [21]:
DF = df([i.to_dict() for i in cct.atoms])
a = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag']]
a

,_atom,basicTag,dictTag,radical,POSTag
0,女,CHN,OTHER,女,n-B
1,性,CHN,OTHER,忄,n-I
2,",",PUNC,OTHER,-,x-B
3,8,NUM,OTHER,-,m-B
4,8,NUM,OTHER,-,m-I
5,岁,CHN,OTHER,山,m-B
6,",",PUNC,OTHER,-,x-B
7,农,CHN,OTHER,冖,n-B
8,民,CHN,OTHER,氏,n-I
9,",",PUNC,OTHER,-,x-B


In [22]:
cols = ['_atom', 'basicTag']

a.to_csv('dev/demo.txt', 
         sep = '\t', 
         encoding = 'utf-8', 
         header = False,
         index = False, 
         columns = cols
        )

### Deal with the CRF Result

In [23]:
outputPath = 'dev/demo_result.txt'
with open(outputPath, 'r') as f:
    for line in f.readlines():
        if '# 0' in line[:3] or '# 1' in f.readlines():
            print(line)
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result

FileNotFoundError: [Errno 2] No such file or directory: 'dev/demo_result.txt'

In [24]:
result.columns = cols + ['Result'] + list(range((len(result.columns)-len(cols) -1)))
result

NameError: name 'result' is not defined

In [44]:
def getRTag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None

a = result['Result'].apply(getRTag)
a

0         O
1         O
2         O
3         O
4         O
5         O
6         O
7         O
8         O
9         O
10        O
11        O
12        O
13        O
14        O
15        O
16        O
17        O
18        O
19        O
20        O
21        O
22        O
23        O
24        O
25        O
26        O
27        O
28        O
29        O
       ... 
352       O
353       O
354     R-B
355     R-I
356     R-I
357     R-I
358     R-I
359     R-I
360       O
361       O
362       O
363       O
364       O
365     R-B
366     R-I
367     R-I
368     R-I
369       O
370       O
371       O
372       O
373     R-B
374     R-I
375     R-I
376     R-I
377       O
378       O
379       O
380       O
381    None
Name: Result, dtype: object

In [45]:
cols =['_atom', 'basicTag']

In [46]:
result['RTag'] = result['Result'].apply(getRTag)
result = result[cols + ['RTag']]
result

,_atom,basicTag,RTag
0,1,NUM,O
1,.,PUNC,O
2,患,CHN,O
3,者,CHN,O
4,中,CHN,O
5,年,CHN,O
6,男,CHN,O
7,性,CHN,O
8,",",PUNC,O
9,慢,CHN,O


In [47]:
newTag = 'RTag'

for idx, row in result.iterrows():
    for a in cct.atoms:
        if a.index == idx and a._atom == row['_atom']:
            a.__setattr__(newTag, row[newTag])
print(idx, row)
a.to_dict()

381 _atom        NaN
basicTag     NaN
RTag        None
Name: 381, dtype: object


{'POSTag': 'x-B',
 '_atom': '。',
 'basicTag': 'PUNC',
 'dictTag': 'OTHER',
 'index': 58,
 'radical': '-'}

In [48]:
a = cct.atoms[0]
a.to_dict()

{'POSTag': 'n-B',
 '_atom': '女',
 'basicTag': 'CHN',
 'dictTag': 'OTHER',
 'index': 0,
 'radical': '女'}

In [49]:
from pandas import read_csv
from pandas import DataFrame as df


def getRTag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None


class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    
    def __init__(self, text):
        # to do: parse the text for this Chinise Clincal Text
        self._text = cleanSpace(strQ2B(text))
        self.atomsBaseList = segmentStr(self._text)
        pass

    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
          
    def setPOSTags(self):
        for a in self.atoms:
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
        
    def updateDF(self):
        'Return an attribute for pandas PF form sentence'
        DF = df([i.to_dict() for i in self.atoms])
        # TODO, may not contain POS Tag
        # OK, actually, it can. hh
        self.dfFormat = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags]
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )
        
    def coporResult(self, result, newTag):
        # The requirements on result
        # it is a dataframe
        # exactly the same as the CCT
        # include the right and same index of CCT's atoms
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getRTag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
                     

In [50]:
#=================
# TEST
#=================
text

'女性，88岁，农民，双滦区应营子村人，主因右髋部摔伤后疼痛肿胀，活动受限5小时于2016-10-29；11：12入院。\n\n\n\n'

In [51]:
cct = ChineseClinicalText(text)
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
cct.updateDF()
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag
0,女,CHN,OTHER,女,n-B
1,性,CHN,OTHER,忄,n-I
2,",",PUNC,OTHER,-,x-B
3,8,NUM,OTHER,-,m-B
4,8,NUM,OTHER,-,m-I
5,岁,CHN,OTHER,山,m-B
6,",",PUNC,OTHER,-,x-B
7,农,CHN,OTHER,冖,n-B
8,民,CHN,OTHER,氏,n-I
9,",",PUNC,OTHER,-,x-B


In [52]:
print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])
print(cct.outputCols)

['_atom', 'basicTag']
['_atom', 'basicTag']


In [53]:
## New we take result as given
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result

,0,1,2,3,4,5
0,1,NUM,O/0.999952,O/0.999952,R-B/0.000029,R-I/0.000020
1,.,PUNC,O/0.999994,O/0.999994,R-B/0.000003,R-I/0.000004
2,患,CHN,O/0.999790,O/0.999790,R-B/0.000203,R-I/0.000007
3,者,CHN,O/0.999898,O/0.999898,R-B/0.000078,R-I/0.000024
4,中,CHN,O/0.999039,O/0.999039,R-B/0.000928,R-I/0.000033
5,年,CHN,O/0.999368,O/0.999368,R-B/0.000163,R-I/0.000469
6,男,CHN,O/0.999866,O/0.999866,R-B/0.000117,R-I/0.000017
7,性,CHN,O/0.999886,O/0.999886,R-B/0.000010,R-I/0.000104
8,",",PUNC,O/1.000000,O/1.000000,R-B/0.000000,R-I/0.000000
9,慢,CHN,O/0.996577,O/0.996577,R-B/0.003418,R-I/0.000005


In [54]:
cct.coporResult(result, 'RTag')
cct.updateDF()
cct.dfFormat

KeyError: "['RTag'] not in index"

In [55]:
print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']


In [56]:
## New we take result as given
outputPath = 'dev/demoRTag_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,NUM,O,O/0.999990,Bo-B/0.000001,Bo-I/0.000001,Ch-B/0.000002,Ch-I/0.000000,O/0.999990,Si-B/0.000001,Si-I/0.000001,Sy-B/0.000001,Sy-I/0.000001,Tr-B/0.000001,Tr-I/0.000001
1,.,PUNC,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
2,患,CHN,O,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
3,者,CHN,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
4,中,CHN,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
5,年,CHN,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
6,男,CHN,O,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
7,性,CHN,O,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
8,",",PUNC,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
9,慢,CHN,O,O/0.999999,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000


In [57]:
cct.coporResult(result, 'ETag')
cct.updateDF()
cct.dfFormat

KeyError: "['RTag' 'ETag'] not in index"

## Deal With Entities

In [58]:
taggedAtoms = [a for a in cct.atoms if a.ETag != 'O']
startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].ETag[-2:] == '-B']
startIdx.append(len(taggedAtoms))
startIdx

AttributeError: 'Atom' object has no attribute 'ETag'

In [41]:
for i in range(len(startIdx)-1):
    entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
    startA, endA = entityAtom[0], entityAtom[-1]
    print([''.join(ea._atom for ea in entityAtom),startA.index, endA.index, startA.ETag.split('-')[0]] )

['右髋部', 21, 23, 'Bo']
['疼痛', 27, 28, 'Sy']
['肿胀', 29, 30, 'Sy']


In [42]:
class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    entities = []
    
    def __init__(self, text):
        # to do: parse the text for this Chinise Clincal Text
        self._text = cleanSpace(strQ2B(text))
        self.atomsBaseList = segmentStr(self._text)
        pass

    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
          
    def setPOSTags(self):
        for a in self.atoms:
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
        
    def updateDF(self):
        'update the attribute for pandas PF form sentence'
        DF = df([i.to_dict() for i in self.atoms])
        # TODO, may not contain POS Tag
        # OK, actually, it can. hh
        self.dfFormat = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags]
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )
        
    def coporResult(self, result, newTag):
        # The requirements on result
        # it is a dataframe
        # exactly the same as the CCT
        # include the right and same index of CCT's atoms
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getRTag)
        result = result[cols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
         
    #### NEW
    #### HERE
    def getEntities(self, eType):
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        return entitiesList

    

In [43]:
a = cct.atoms[0]
a.get_attr('RTag')

'O'

In [44]:
text
cct = ChineseClinicalText(text)
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
cct.updateDF()
cct.dfFormat
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
cct.coporResult(result, 'RTag')
cct.updateDF()
cct.dfFormat
print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)
outputPath = 'dev/demoRTag_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result
cct.coporResult(result, 'ETag')
cct.updateDF()
cct.dfFormat

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']


,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,女,CHN,OTHER,女,n-B,O,O
1,性,CHN,OTHER,忄,n-I,O,O
2,",",PUNC,OTHER,-,x-B,O,O
3,8,NUM,OTHER,-,m-B,O,O
4,8,NUM,OTHER,-,m-I,O,O
5,岁,CHN,OTHER,山,m-B,O,O
6,",",PUNC,OTHER,-,x-B,O,O
7,农,CHN,OTHER,冖,n-B,O,O
8,民,CHN,OTHER,氏,n-I,O,O
9,",",PUNC,OTHER,-,x-B,O,O


In [45]:
print(cct.getEntities('RTag'))
print(cct.getEntities('ETag'))

[['右髋部', 21, 23, 'R'], ['疼痛', 27, 28, 'R'], ['肿胀', 29, 30, 'R']]
[['右髋部', 21, 23, 'Bo'], ['疼痛', 27, 28, 'Sy'], ['肿胀', 29, 30, 'Sy']]


### Adding Annotated Entities

In [46]:
fLabel = {'症状和体征': 'Sy',
          '身体部位': 'Bo',
          '检查和检验': 'Ch',
          '治疗': 'Tr',
          '疾病和诊断': 'Si'}

In [47]:
annotedEntities = []
with open('dev/一般项目-1.txt', 'r') as f:
    for l in f.readlines():
        entity = strQ2B(l).replace('\n','').split('\t')
        entity[-1] = fLabel[entity[-1]]
        entity[1], entity[2] = int(entity[1]), int(entity[2])
        annotedEntities.append(entity)
        
annotedEntities

[['右髋部', 21, 23, 'Bo'], ['疼痛', 27, 28, 'Sy'], ['肿胀', 29, 30, 'Sy']]

In [48]:
for e in annotedEntities:
    e[-1] = 'R'
annotedEntities

[['右髋部', 21, 23, 'R'], ['疼痛', 27, 28, 'R'], ['肿胀', 29, 30, 'R']]

In [49]:
cct.getEntities('ETag')

[['右髋部', 21, 23, 'Bo'], ['疼痛', 27, 28, 'Sy'], ['肿胀', 29, 30, 'Sy']]

## Final Version

In [50]:
class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    entities = []
    learnedEntities = []
    annotedEntities = []
    annotedFilePath = None
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))
                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')


    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
          
    def setPOSTags(self):
        for a in self.atoms:
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
        
    def updateDF(self):
        'update the attribute for pandas PF form sentence'
        DF = df([i.to_dict() for i in self.atoms])
        # TODO, may not contain POS Tag
        # OK, actually, it can. hh
        self.dfFormat = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags]
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )
        
    def coporResult(self, result, newTag):
        # The requirements on result
        # it is a dataframe
        # exactly the same as the CCT
        # include the right and same index of CCT's atoms
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getRTag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
        self.updateDF()
        
    
         
    #### NEW
    #### HERE
    def getLearnedEntities(self, eType):
        '''eType = 'RTag' or 'ETag'''
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        entitiesList = [tuple(i) for i in entitiesList]
        self.learnedEntities = entitiesList
        return entitiesList

    def getAnnotedEntities(self, eType, annotedFilePath = None):
        '''eType = 'RTag' or 'ETag'''
        if annotedFilePath == None:
            annotedFilePath = self.annotedFilePath
        annotedEntities = []
        with open(annotedFilePath, 'r') as f:
            for l in f.readlines():
                entity = strQ2B(l).replace('\n','').split('\t')
                entity[-1] = fLabel[entity[-1]]
                entity[1], entity[2] = int(entity[1]), int(entity[2])
                annotedEntities.append(entity)
        if eType == 'RTag':
            for ae in annotedEntities:
                ae[-1] = 'R'
            self.annotedREntities = [tuple(i) for i in annotedEntities]
        
        annotedEntities = [tuple(i) for i in annotedEntities]
        self.annotedEntities = annotedEntities
        return annotedEntities
    
    def data(self):
        s = {}
        # Num of Atoms
        s['atoms_n'] = len(self.atoms)
        s['learned_n'] = len(self.learnedEntities)
        s['annoted_n'] = len(self.annotedEntities)
        s['matched_n'] = len(set(self.learnedEntities).intersection(set(self.annotedEntities)))
        return s
    
    def execute(self):
        self.setAtomTags()   # set atoms' tags
        self.getPOSTags()    # generate atoms' POS tags. (only valid in text)
        self.setPOSTags()    # set atoms' POS tags
        self.updateDF()      # update atoms' attributes to DF format


In [51]:
orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'

cct = ChineseClinicalText(originalFilePath= orig, annotedFilePath= anno)
cct.execute()

# Take as granted
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)



In [52]:
orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'

cct = ChineseClinicalText(originalFilePath= orig, annotedFilePath= anno)
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
cct.updateDF()
# You can see some output here.
cct.dfFormat

# This file is intend to be annoted by model
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])

# Take as granted
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)


cct.coporResult(result, 'RTag')
#cct.updateDF()
cct.dfFormat


print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)
outputPath = 'dev/demoRTag_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)

cct.coporResult(result, 'ETag')
# cct.updateDF()
cct.dfFormat

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']


,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,女,CHN,OTHER,女,n-B,O,O
1,性,CHN,OTHER,忄,n-I,O,O
2,",",PUNC,OTHER,-,x-B,O,O
3,8,NUM,OTHER,-,m-B,O,O
4,8,NUM,OTHER,-,m-I,O,O
5,岁,CHN,OTHER,山,m-B,O,O
6,",",PUNC,OTHER,-,x-B,O,O
7,农,CHN,OTHER,冖,n-B,O,O
8,民,CHN,OTHER,氏,n-I,O,O
9,",",PUNC,OTHER,-,x-B,O,O


In [53]:
cct.getAnnotedEntities('ETag')
cct.getLearnedEntities('ETag')

a = cct.annotedEntities
l = cct.learnedEntities
print(a)
print(l)
cct.data()

[('右髋部', 21, 23, 'Bo'), ('疼痛', 27, 28, 'Sy'), ('肿胀', 29, 30, 'Sy')]
[('右髋部', 21, 23, 'Bo'), ('疼痛', 27, 28, 'Sy'), ('肿胀', 29, 30, 'Sy')]


{'annoted_n': 3, 'atoms_n': 59, 'learned_n': 3, 'matched_n': 3}

### Final Test (Psudo)

In [54]:
from text import ChineseClinicalText as CCT
from pandas import read_csv


orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'


In [55]:
from text import ChineseClinicalText as CCT
from pandas import read_csv


orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'

cct = CCT(originalFilePath= orig, annotedFilePath= anno)
cct.setAtomTags()   # set atoms' tags
cct.getPOSTags()    # generate atoms' POS tags. (only valid in text)
cct.setPOSTags()    # set atoms' POS tags
cct.updateDF()      # update atoms' attributes to DF format
# You can see some output here.
cct.dfFormat        # the updated DF format

# This file is intend to be annoted by model
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])

# Take as granted
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)


cct.coporResult(result, 'RTag')
cct.updateDF()
cct.dfFormat


print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)
outputPath = 'dev/demoRTag_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)

cct.coporResult(result, 'ETag')
cct.updateDF()
cct.dfFormat
cct.getAnnotedEntities('ETag')
cct.getLearnedEntities('ETag')
cct.data()

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']


{'Atoms': 59,
 'Bo_annoted': 1,
 'Bo_learned': 1,
 'Bo_matched': 1,
 'Ch_annoted': 0,
 'Ch_learned': 0,
 'Ch_matched': 0,
 'Si_annoted': 0,
 'Si_learned': 0,
 'Si_matched': 0,
 'Sy_annoted': 2,
 'Sy_learned': 2,
 'Sy_matched': 2,
 'Tr_annoted': 0,
 'Tr_learned': 0,
 'Tr_matched': 0,
 'annoted_E': 3,
 'annoted_R': 0,
 'learned_E': 3,
 'learned_R': 0,
 'matched_E': 3,
 'matched_R': 0}

## For Test Set

In [56]:
from text import ChineseClinicalText as CCT
from pandas import read_csv


orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'

cct = CCT(originalFilePath= orig, annotedFilePath= anno)
cct.execute()

# This file is intend to be annoted by model
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])

###################
# Take as granted
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
###################

cct.coporResult(result, 'RTag')
#cct.updateDF()
#cct.dfFormat

print(cct.outputCols)
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)

###################
# Take as granted
outputPath = 'dev/demoRTag_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
###################


cct.coporResult(result, 'ETag')
#cct.updateDF()
#cct.dfFormat
print(cct.getAnnotedEntities('ETag'))
print(cct.getLearnedEntities('ETag'))
cct.data()

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']
[['右髋部', 21, 23, 'Bo'], ['疼痛', 27, 28, 'Sy'], ['肿胀', 29, 30, 'Sy']]
[('右髋部', 21, 23, 'Bo'), ('疼痛', 27, 28, 'Sy'), ('肿胀', 29, 30, 'Sy')]


{'Atoms': 59,
 'Bo_annoted': 1,
 'Bo_learned': 1,
 'Bo_matched': 1,
 'Ch_annoted': 0,
 'Ch_learned': 0,
 'Ch_matched': 0,
 'Si_annoted': 0,
 'Si_learned': 0,
 'Si_matched': 0,
 'Sy_annoted': 2,
 'Sy_learned': 2,
 'Sy_matched': 2,
 'Tr_annoted': 0,
 'Tr_learned': 0,
 'Tr_matched': 0,
 'annoted_E': 3,
 'annoted_R': 0,
 'learned_E': 3,
 'learned_R': 0,
 'matched_E': 3,
 'matched_R': 0}

### 蜜汁问题

In [57]:
from text import ChineseClinicalText as CCT
from pandas import read_csv

####Prepare Input
outputPath = 'dev/demo_result.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)

In [58]:
from text import ChineseClinicalText as CCT
from pandas import read_csv

orig = 'dev/一般项目-1.txtoriginal.txt'
anno = 'dev/一般项目-1.txt'
cct = CCT(originalFilePath= orig, annotedFilePath= anno)
cct.newTags


[]

In [59]:
cct.execute()

# Output, then Input
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])
print(cct.outputCols)



['_atom', 'basicTag']


In [60]:
# Output, then Input
cct.toTextFile('dev/demoRTag.txt', cols = ['_atom', 'basicTag'])
print(cct.outputCols)

cct.coporResult(result, 'RTag')
cct.dfFormat

['_atom', 'basicTag']


,_atom,basicTag,dictTag,radical,POSTag,RTag
0,女,CHN,OTHER,女,n-B,O
1,性,CHN,OTHER,忄,n-I,O
2,",",PUNC,OTHER,-,x-B,O
3,8,NUM,OTHER,-,m-B,O
4,8,NUM,OTHER,-,m-I,O
5,岁,CHN,OTHER,山,m-B,O
6,",",PUNC,OTHER,-,x-B,O
7,农,CHN,OTHER,冖,n-B,O
8,民,CHN,OTHER,氏,n-I,O
9,",",PUNC,OTHER,-,x-B,O


In [61]:
cct = CCT(originalFilePath= orig, annotedFilePath= anno)
cct.__dict__
cct.newTags

[]

In [62]:
cct.setAtomTags()   # set atoms' tags
cct.getPOSTags()    # generate atoms' POS tags. (only valid in text)
cct.setPOSTags()    # set atoms' POS tags
#cct.updateDF()      # update atoms' attributes to DF format

In [63]:
cct.newTags

[]

In [64]:
cct = CCT(originalFilePath= orig, annotedFilePath= anno)
cct.__dict__
cct.newTags

[]

### New Problem, On Test Data


In [65]:
anE = cct.annotedEntities
entityTagsList = []
for ae in anE:
    word = ae[0]
    for i in range(len(word)):
        if i == 0:
            entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
        else:
            entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
entityTagsList

[]

In [66]:
for a in cct.atoms:
    a.set_attr(newTag, 'O')
    for eTag in entityTagsList:
        if a.index == eTag[0] and a._atom == eTag[1]:
            a.set_attr(newTag, eTag[-1])

# Final Version

In [40]:
from pandas import read_csv
from pandas import DataFrame as df
from jieba import posseg as ppseg
import string

from atom import Atom

# this dictionary is used to deal with the annotated entities in texts.
fLabel = {'症状和体征': 'Sy',
          '身体部位':   'Bo',
          '检查和检验': 'Ch',
          '治疗':      'Tr',
          '疾病和诊断': 'Si'}


def strQ2B(ustring):
    rstring = ''
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):
            inside_code -= 65248
        # 2: unichr; 3: chr
        rstring += chr(inside_code)
    return rstring


def cleanSpace(text):
    #return text.replace("\t"," ").replace(" ","&nbsp;").replace('\n', '')
    '''
    return text.strip()
    '''
    for i in ' \t\r\x0b\x0c':
        text = text.replace(i, '@')
    text = text.strip()
    text =  text.replace('\n', '@')
    return text 
    

def segmentStr(text):
    strLen = len(text)
    atoms = []
    for i in range(strLen):
        atoms.append(text[i])
    return atoms


def getPOSTagsList(text):
    segs = list(ppseg.cut(text))
    
    for i in range(len(segs)):
        pair = segs[i]
        start = sum(len(p.word) for p in segs[:i])
        end = sum(len(p.word) for p in segs[:i+1]) -1
        pair.indeces = [start, end]
        
    POSTagsList = []
    for p in segs:
        word = p.word
        for i in range(len(p.word)):
            if i == 0:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
            else:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
                
    return POSTagsList

def getRTag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None


class ChineseClinicalText(object):
    atoms = []
    POSTags = None
    outputCols =['_atom', 'basicTag']
    newTags = []
    entities = []
    learnedEntities = []
    annotedEntities = []
    learnedREntities = []
    annotedREntities = []
    annotedFilePath = None
    score = None
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')

    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
          
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
        
    def updateDF(self):
        'update the attribute for pandas PF form sentence'
        DF = df([i.to_dict() for i in self.atoms])
        # TODO, may not contain POS Tag
        # OK, actually, it can. hh
        self.dfFormat = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags]
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )
        
    # For Test Data, need Copor Result Data
    def coporResult(self, result, newTag):
        # The requirements on result
        # it is a dataframe
        # exactly the same as the CCT
        # include the right and same index of CCT's atoms
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getRTag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
        self.updateDF()
        
    # For Train Data, need Copor Annotated Data
    def coporAnnotation(self, newTag):
        entityTagsList = []
        if newTag == 'RTag':
            for ae in self.annotedREntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        elif newTag == 'ETag':
            for ae in self.annotedEntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        for a in self.atoms:
            a.set_attr(newTag, 'O')
            for eTag in entityTagsList:
                if a.index == eTag[0] and a._atom == eTag[1]:
                    a.set_attr(newTag, eTag[-1])
    
        self.newTags.append(newTag)
        self.updateDF()
         
    #### NEW
    #### HERE
    def getLearnedEntities(self, eType):
        '''
        eType = 'RTag' or 'ETag
        Atoms have 'RTag' or 'ETag'
        '''
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        entitiesList = [tuple(i) for i in entitiesList]
        if eType == 'RTag':
            self.learnedREntities = entitiesList
        if eType == 'ETag':
            self.learnedEntities  = entitiesList
        return entitiesList

    def getAnnotedEntities(self, eType, annotedFilePath = None):
        '''
        eType = 'RTag' or 'ETag'
        '''
        if annotedFilePath == None:
            annotedFilePath = self.annotedFilePath
        annotedEntities = []
        with open(annotedFilePath, 'r') as f:
            for l in f.readlines():
                entity = strQ2B(l).replace('\n','').split('\t')
                entity[0] = cleanSpace(entity[0])
                entity[-1] = fLabel[entity[-1]]
                entity[1], entity[2] = int(entity[1]), int(entity[2])
                annotedEntities.append(entity)

        #New here, Add This to check the text is consistent or not
        
        for ae in annotedEntities:
            if self._text[ae[1]:ae[2]+1] != ae[0]:
                print('Not Macthed:', self.originalFilePath)
                break
        
        #New here--END

        if eType == 'RTag':
            for ae in annotedEntities:
                ae[-1] = 'R'
            self.annotedREntities = [tuple(i) for i in annotedEntities]
        elif eType == 'ETag':
            self.annotedEntities  = [tuple(i) for i in annotedEntities]

        return annotedEntities
    
    def data(self):
        s = {}
        # Num of Atoms
        s['Atoms'] = len(self.atoms)
        s['learned_E'] = len(self.learnedEntities)
        s['annoted_E'] = len(self.annotedEntities)
        s['matched_E'] = len(set(self.learnedEntities).intersection(set(self.annotedEntities)))## Union vs Join
        s['learned_R'] = len(self.learnedREntities)
        s['annoted_R'] = len(self.annotedREntities)
        s['matched_R'] = len(set(self.learnedREntities).intersection(set(self.annotedREntities)))## Union vs Join
        entitiesLable = ['Sy','Bo', 'Ch', 'Tr', 'Si']
        for eL in entitiesLable:
            elL = [e for e in self.learnedEntities if eL == e[-1]]
            elA = [e for e in self.annotedEntities if eL == e[-1]]
            elM = set(elA).intersection(set(elL)) ## Union vs Join
            s[eL+'_learned'] = len(elL)
            s[eL+'_annoted'] = len(elA)
            s[eL+'_matched'] = len(elM)
        
        return s
    
    def execute(self):
        self.setAtomTags()   # set atoms' tags
        self.getPOSTags()    # generate atoms' POS tags. (only valid in text)
        self.setPOSTags()    # set atoms' POS tags
        self.updateDF()      # update atoms' attributes to DF format





## For Train Set

In [42]:
cct.getAnnotedEntities('RTag')
cct.coporAnnotation('RTag')
cct.getAnnotedEntities('ETag')
cct.coporAnnotation('ETag')

etime = time.clock()
print(etime - btime)

cct.toTextFile(devPath + 'R' + anno, cols = ['_atom', 'basicTag', 'RTag'])
cct.toTextFile(devPath + 'E' + anno, cols = ['_atom', 'basicTag', 'RTag', 'ETag'])

# THIS PART BELONGS TO TEST DATASET
#cct.coporResult(result, 'ETag')
#cct.updateDF()
#cct.dfFormat

print(cct.getAnnotedEntities('ETag'))
#print(cct.getLearnedEntities('ETag'))
#cct.data()

1.5820149999999993
[['腰痛', 35, 36, 'Sy'], ['左下肢', 38, 40, 'Bo'], ['咳嗽', 48, 49, 'Sy'], ['疼痛', 73, 74, 'Sy'], ['腰椎间盘CT', 102, 107, 'Ch'], ['L3-S1椎间盘CT平扫', 111, 122, 'Ch'], ['腰椎', 125, 126, 'Bo'], ['L3-5椎间盘', 134, 140, 'Bo'], ['椎体', 142, 143, 'Bo'], ['膨大', 146, 147, 'Sy'], ['L5-S1椎间盘', 159, 166, 'Bo'], ['椎管', 171, 172, 'Bo'], ['硬膜囊', 177, 179, 'Bo'], ['神经根', 181, 183, 'Bo'], ['椎体边缘', 187, 190, 'Bo'], ['椎小关节', 192, 195, 'Bo'], ['腰椎管', 207, 209, 'Bo'], ['椎间孔', 211, 213, 'Bo'], ['侧隐窝', 215, 217, 'Bo'], ['狭窄', 218, 219, 'Sy'], ['周围软组织', 221, 225, 'Bo'], ['查体', 261, 262, 'Ch'], ['T', 264, 264, 'Ch'], ['P', 272, 272, 'Ch'], ['R', 280, 280, 'Ch'], ['BP', 288, 289, 'Ch'], ['心', 307, 307, 'Bo'], ['肺', 308, 308, 'Bo'], ['腹', 309, 309, 'Bo'], ['腰椎', 322, 323, 'Bo'], ['L4-S1椎体棘突', 333, 341, 'Bo'], ['压痛', 345, 346, 'Ch'], ['直腿抬高试验', 354, 359, 'Ch'], ['拾物试验', 365, 368, 'Ch'], ['4字试验', 373, 376, 'Ch']]


In [7]:
import time
btime = time.clock()
from pandas import read_csv
# from text import ChineseClinicalText

devPath = 'dev/Text/'
anno = '病史特点-221.txt'
orig = '病史特点-221.txtoriginal.txt'

anno = '一般项目-5.txt'
orig = '一般项目-5.txtoriginal.txt'

cct = ChineseClinicalText(originalFilePath= devPath + orig, 
                          annotedFilePath = devPath + anno)

In [8]:
cct.execute()
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag
0,老,CHN,OTHER,老,t-B
1,年,CHN,OTHER,干,t-I
2,女,CHN,OTHER,女,n-B
3,性,CHN,OTHER,忄,n-I
4,",",PUNC,OTHER,-,x-B
5,7,NUM,OTHER,-,m-B
6,0,NUM,OTHER,-,m-I
7,岁,CHN,OTHER,山,m-B
8,。,PUNC,OTHER,-,x-B
9,籍,CHN,OTHER,竹,nr-B


In [9]:
cct.getAnnotedEntities('RTag')
cct.coporAnnotation('RTag')
cct.getAnnotedEntities('ETag')
cct.coporAnnotation('ETag')
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,老,CHN,OTHER,老,t-B,O,O
1,年,CHN,OTHER,干,t-I,O,O
2,女,CHN,OTHER,女,n-B,O,O
3,性,CHN,OTHER,忄,n-I,O,O
4,",",PUNC,OTHER,-,x-B,O,O
5,7,NUM,OTHER,-,m-B,O,O
6,0,NUM,OTHER,-,m-I,O,O
7,岁,CHN,OTHER,山,m-B,O,O
8,。,PUNC,OTHER,-,x-B,O,O
9,籍,CHN,OTHER,竹,nr-B,O,O


In [63]:
cct.toTextFile(devPath + 'R' + anno, cols = ['_atom', 'basicTag', 'RTag'])
cct.toTextFile(devPath + 'E' + anno, cols = ['_atom', 'basicTag', 'RTag', 'ETag'])

In [65]:
cct.getAnnotedEntities('ETag')

[['腰痛', 35, 36, 'Sy'],
 ['左下肢', 38, 40, 'Bo'],
 ['咳嗽', 48, 49, 'Sy'],
 ['疼痛', 73, 74, 'Sy'],
 ['腰椎间盘CT', 102, 107, 'Ch'],
 ['L3-S1椎间盘CT平扫', 111, 122, 'Ch'],
 ['腰椎', 125, 126, 'Bo'],
 ['L3-5椎间盘', 134, 140, 'Bo'],
 ['椎体', 142, 143, 'Bo'],
 ['膨大', 146, 147, 'Sy'],
 ['L5-S1椎间盘', 159, 166, 'Bo'],
 ['椎管', 171, 172, 'Bo'],
 ['硬膜囊', 177, 179, 'Bo'],
 ['神经根', 181, 183, 'Bo'],
 ['椎体边缘', 187, 190, 'Bo'],
 ['椎小关节', 192, 195, 'Bo'],
 ['腰椎管', 207, 209, 'Bo'],
 ['椎间孔', 211, 213, 'Bo'],
 ['侧隐窝', 215, 217, 'Bo'],
 ['狭窄', 218, 219, 'Sy'],
 ['周围软组织', 221, 225, 'Bo'],
 ['查体', 261, 262, 'Ch'],
 ['T', 264, 264, 'Ch'],
 ['P', 272, 272, 'Ch'],
 ['R', 280, 280, 'Ch'],
 ['BP', 288, 289, 'Ch'],
 ['心', 307, 307, 'Bo'],
 ['肺', 308, 308, 'Bo'],
 ['腹', 309, 309, 'Bo'],
 ['腰椎', 322, 323, 'Bo'],
 ['L4-S1椎体棘突', 333, 341, 'Bo'],
 ['压痛', 345, 346, 'Ch'],
 ['直腿抬高试验', 354, 359, 'Ch'],
 ['拾物试验', 365, 368, 'Ch'],
 ['4字试验', 373, 376, 'Ch']]

In [66]:
# 这里意义不大
cct.data()

{'Atoms': 381,
 'Bo_annoted': 19,
 'Bo_learned': 0,
 'Bo_matched': 0,
 'Ch_annoted': 11,
 'Ch_learned': 0,
 'Ch_matched': 0,
 'Si_annoted': 0,
 'Si_learned': 0,
 'Si_matched': 0,
 'Sy_annoted': 5,
 'Sy_learned': 0,
 'Sy_matched': 0,
 'Tr_annoted': 0,
 'Tr_learned': 0,
 'Tr_matched': 0,
 'annoted_E': 35,
 'annoted_R': 35,
 'learned_E': 0,
 'learned_R': 0,
 'matched_E': 0,
 'matched_R': 0}

## For Test Set

In [3]:
from text import ChineseClinicalText

from pandas import read_csv

anno = 'dev/Text/一般项目-5.txt'
orig = 'dev/Text/一般项目-5.txtoriginal.txt'

cct = ChineseClinicalText(originalFilePath= orig, annotedFilePath= anno)
cct.execute()

# This file is intend to be annoted by model
cct.toTextFile('dev/Text/2ab病史特点-61-1Test.txt', cols = ['_atom', 'basicTag'])

###################
# Take as granted
outputPath = 'dev/Text/2ab病史特点-61-1Rslt.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
###################

cct.coporResult(result, 'RTag')
#cct.updateDF()
#cct.dfFormat

print(cct.outputCols)
cct.toTextFile('dev/Text/2ab病史特点-61-2Test.txt', cols = ['_atom', 'basicTag', 'RTag'])
print(cct.outputCols)

###################
# Take as granted
outputPath = 'dev/Text/2ab病史特点-61-2Rslt.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
###################


cct.coporResult(result, 'ETag')
#cct.updateDF()
#cct.dfFormat
cct.getAnnotedEntities('RTag')
cct.getLearnedEntities('RTag')
cct.getAnnotedEntities('ETag')
cct.getLearnedEntities('ETag')

cct.data()

['_atom', 'basicTag']
['_atom', 'basicTag', 'RTag']


AttributeError: 'Atom' object has no attribute 'RTag'

In [4]:
anno = 'dev/Text/一般项目-5.txt'
orig = 'dev/Text/一般项目-5.txtoriginal.txt'
cct = ChineseClinicalText(originalFilePath= orig, annotedFilePath= anno)

In [5]:
cct.execute()
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag
0,老,CHN,OTHER,老,t-B
1,年,CHN,OTHER,干,t-I
2,女,CHN,OTHER,女,n-B
3,性,CHN,OTHER,忄,n-I
4,",",PUNC,OTHER,-,x-B
5,7,NUM,OTHER,-,m-B
6,0,NUM,OTHER,-,m-I
7,岁,CHN,OTHER,山,m-B
8,。,PUNC,OTHER,-,x-B
9,籍,CHN,OTHER,竹,nr-B


In [80]:
# This file is intend to be annoted by model
cct.toTextFile('dev/Text/2ab病史特点-61-1Test.txt', cols = ['_atom', 'basicTag'])

In [81]:
###################
# Take as granted
outputPath = 'dev/Text/2ab病史特点-61-1Rslt.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result
###################

,0,1,2,3,4,5
0,1,NUM,O/0.999997,O/0.999997,R-B/0.000002,R-I/0.000001
1,.,PUNC,O/1.000000,O/1.000000,R-B/0.000000,R-I/0.000000
2,患,CHN,O/0.999997,O/0.999997,R-B/0.000003,R-I/0.000000
3,者,CHN,O/0.999997,O/0.999997,R-B/0.000002,R-I/0.000000
4,",",PUNC,O/1.000000,O/1.000000,R-B/0.000000,R-I/0.000000
5,女,CHN,O/0.999227,O/0.999227,R-B/0.000772,R-I/0.000001
6,",",PUNC,O/1.000000,O/1.000000,R-B/0.000000,R-I/0.000000
7,6,NUM,O/0.999998,O/0.999998,R-B/0.000002,R-I/0.000000
8,8,NUM,O/0.999999,O/0.999999,R-B/0.000000,R-I/0.000000
9,岁,CHN,O/0.999977,O/0.999977,R-B/0.000022,R-I/0.000001


In [82]:
cct.coporResult(result, 'RTag')
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag,RTag
0,1,NUM,OTHER,-,m-B,O
1,.,PUNC,OTHER,-,m-I,O
2,患,CHN,OTHER,心,n-B,O
3,者,CHN,OTHER,耂,n-I,O
4,",",PUNC,OTHER,-,x-B,O
5,女,CHN,OTHER,女,b-B,O
6,",",PUNC,OTHER,-,x-B,O
7,6,NUM,OTHER,-,m-B,O
8,8,NUM,OTHER,-,m-I,O
9,岁,CHN,OTHER,山,m-B,O


In [83]:
cct.toTextFile('dev/Text/2ab病史特点-61-2Test.txt', cols = ['_atom', 'basicTag', 'RTag'])

In [84]:
###################
# Take as granted
outputPath = 'dev/Text/2ab病史特点-61-2Rslt.txt'
result = read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result
###################

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,NUM,O,O/0.999955,Bo-B/0.000003,Bo-I/0.000008,Ch-B/0.000001,Ch-I/0.000001,O/0.999955,Si-B/0.000003,Si-I/0.000003,Sy-B/0.000016,Sy-I/0.000004,Tr-B/0.000003,Tr-I/0.000003
1,.,PUNC,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
2,患,CHN,O,O/0.999999,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
3,者,CHN,O,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
4,",",PUNC,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
5,女,CHN,O,O/0.999998,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999998,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
6,",",PUNC,O,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
7,6,NUM,O,O/0.999997,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000001,Ch-I/0.000000,O/0.999997,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
8,8,NUM,O,O/0.999997,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000001,Ch-I/0.000000,O/0.999997,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000
9,岁,CHN,O,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000


In [85]:
cct.coporResult(result, 'ETag')
cct.dfFormat

,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,1,NUM,OTHER,-,m-B,O,O
1,.,PUNC,OTHER,-,m-I,O,O
2,患,CHN,OTHER,心,n-B,O,O
3,者,CHN,OTHER,耂,n-I,O,O
4,",",PUNC,OTHER,-,x-B,O,O
5,女,CHN,OTHER,女,b-B,O,O
6,",",PUNC,OTHER,-,x-B,O,O
7,6,NUM,OTHER,-,m-B,O,O
8,8,NUM,OTHER,-,m-I,O,O
9,岁,CHN,OTHER,山,m-B,O,O


In [86]:
cct.getAnnotedEntities('RTag')
cct.getLearnedEntities('RTag')
cct.getAnnotedEntities('ETag')

[['骨折', 25, 26, 'Sy'],
 ['头部', 52, 53, 'Bo'],
 ['前胸', 55, 56, 'Bo'],
 ['后背', 58, 59, 'Bo'],
 ['头部', 61, 62, 'Bo'],
 ['疼痛', 63, 64, 'Sy'],
 ['出血', 65, 66, 'Sy'],
 ['疼痛', 70, 71, 'Sy'],
 ['头痛', 73, 74, 'Sy'],
 ['头晕', 75, 76, 'Sy'],
 ['恶心', 85, 86, 'Sy'],
 ['呕吐', 87, 88, 'Sy'],
 ['胸闷', 91, 92, 'Sy'],
 ['心悸', 93, 94, 'Sy'],
 ['二便', 97, 98, 'Bo'],
 ['T', 112, 112, 'Ch'],
 ['P', 120, 120, 'Ch'],
 ['R', 128, 128, 'Ch'],
 ['血压', 136, 137, 'Ch'],
 ['查体', 162, 163, 'Ch'],
 ['结膜', 167, 168, 'Bo'],
 ['苍白', 170, 171, 'Sy'],
 ['巩膜', 173, 174, 'Bo'],
 ['黄染', 176, 177, 'Sy'],
 ['口唇', 179, 180, 'Bo'],
 ['发绀', 182, 183, 'Sy'],
 ['气管', 192, 193, 'Bo'],
 ['甲状腺', 197, 199, 'Bo'],
 ['肿大', 201, 202, 'Sy'],
 ['胸廓', 204, 205, 'Bo'],
 ['畸形', 209, 210, 'Sy'],
 ['呼吸动度', 212, 215, 'Ch'],
 ['双侧语颤', 228, 231, 'Ch'],
 ['皮下捻发音', 242, 246, 'Ch'],
 ['双肺', 248, 249, 'Bo'],
 ['叩音', 250, 251, 'Ch'],
 ['肺', 254, 254, 'Bo'],
 ['肝', 255, 255, 'Bo'],
 ['相对浊音界', 256, 260, 'Ch'],
 ['右侧锁骨中线第Ⅴ肋间', 263, 272, 'Bo'],
 ['双肺', 274, 275

In [87]:
cct.getLearnedEntities('ETag')

[('骨折', 25, 26, 'Sy'),
 ('头部', 52, 53, 'Bo'),
 ('前胸', 55, 56, 'Bo'),
 ('后背', 58, 59, 'Bo'),
 ('头部', 61, 62, 'Bo'),
 ('疼痛', 63, 64, 'Sy'),
 ('出血', 65, 66, 'Sy'),
 ('疼痛', 70, 71, 'Sy'),
 ('头痛', 73, 74, 'Sy'),
 ('头晕', 75, 76, 'Sy'),
 ('恶心', 85, 86, 'Sy'),
 ('呕吐', 87, 88, 'Sy'),
 ('胸闷', 91, 92, 'Sy'),
 ('心悸', 93, 94, 'Sy'),
 ('二便', 97, 98, 'Bo'),
 ('T', 112, 112, 'Ch'),
 ('P', 120, 120, 'Ch'),
 ('R', 128, 128, 'Ch'),
 ('血压', 136, 137, 'Ch'),
 ('查体', 162, 163, 'Ch'),
 ('结膜', 167, 168, 'Bo'),
 ('苍白', 170, 171, 'Sy'),
 ('巩膜', 173, 174, 'Bo'),
 ('黄染', 176, 177, 'Sy'),
 ('口唇', 179, 180, 'Bo'),
 ('发绀', 182, 183, 'Sy'),
 ('气管', 192, 193, 'Bo'),
 ('甲状腺', 197, 199, 'Bo'),
 ('肿大', 201, 202, 'Sy'),
 ('胸廓', 204, 205, 'Bo'),
 ('畸形', 209, 210, 'Sy'),
 ('呼吸动度', 212, 215, 'Ch'),
 ('双侧语颤', 228, 231, 'Ch'),
 ('皮下捻发音', 242, 246, 'Ch'),
 ('双肺', 248, 249, 'Bo'),
 ('叩音', 250, 251, 'Ch'),
 ('肺', 254, 254, 'Bo'),
 ('肝', 255, 255, 'Bo'),
 ('相对浊音界', 256, 260, 'Ch'),
 ('右侧锁骨中线第Ⅴ肋间', 263, 272, 'Bo'),
 ('双肺', 274, 275

In [88]:
cct.data()

{'Atoms': 548,
 'Bo_annoted': 35,
 'Bo_learned': 34,
 'Bo_matched': 34,
 'Ch_annoted': 25,
 'Ch_learned': 25,
 'Ch_matched': 24,
 'Si_annoted': 0,
 'Si_learned': 1,
 'Si_matched': 0,
 'Sy_annoted': 26,
 'Sy_learned': 26,
 'Sy_matched': 26,
 'Tr_annoted': 0,
 'Tr_learned': 0,
 'Tr_matched': 0,
 'annoted_E': 86,
 'annoted_R': 86,
 'learned_E': 86,
 'learned_R': 86,
 'matched_E': 84,
 'matched_R': 84}